<a href="https://colab.research.google.com/github/Munya07/NLP/blob/master/2.NER/Elmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Keras==1.0.6
import numpy as np
from keras.layers.recurrent import LSTM
from keras.layers.core import  Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import tensorflow_hub as hub
from keras.layers import Dense,TimeDistributed
import keras.layers as layers
from keras.models import Model,Sequential
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import *

Using TensorFlow backend.
W0606 06:57:35.915053 139849357653888 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
raw = open('drive/My Drive/elmo/wikigold.conll.txt', 'r').readlines()
 
all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(' ')
    point.append(stripped_line)
    if line == '\n':
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]
 
lengths = [len(x) for x in all_x]
print('Input sequence length range: ', max(lengths), min(lengths))
 
short_x = [x for x in all_x if len(x) < 64]
 
X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]
 
all_text = [c for x in X for c in x]


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Input sequence length range:  144 1


In [3]:
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}
print('Vocabulary size:', len(word2ind), len(label2ind))

Vocabulary size: 8285 5


In [0]:
maxlen = max([len(x) for x in X])
print('Maximum sequence length:', maxlen)
print(label2ind)

In [0]:
def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result
 
X_enc = [[word2ind[c] for c in x] for x in X]
max_label = max(label2ind.values()) + 1
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]
 
X_enc = pad_sequences(X_enc, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=11*32, train_size=45*32, random_state=42)
print('Training and testing tensor shapes:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)
 
max_features = len(word2ind)
embedding_size = 300
hidden_size = 32
out_size = len(label2ind) + 1
batch_size = 32

Training and testing tensor shapes: (1440, 63) (352, 63) (1440, 63, 6) (352, 63, 6)


In [7]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

Instructions for updating:
Colocations handled automatically by placer.


W0606 06:59:25.732404 139849357653888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [0]:

class ELMoEmbedding(Layer):

    def __init__(self, idx2word, output_mode="default", trainable=True, **kwargs):
        assert output_mode in ["default", "word_emb", "lstm_outputs1", "lstm_outputs2", "elmo"]
        assert trainable in [True, False]
        self.idx2word = idx2word
        self.output_mode = output_mode
        self.trainable = trainable
        self.max_length = None
        self.word_mapping = None
        self.lookup_table = None
        self.elmo_model = None
        self.embedding = None
        super(ELMoEmbedding, self).__init__(**kwargs)

    def build(self, input_shape):
        self.max_length = input_shape[1]
        self.word_mapping = [x[1] for x in sorted(self.idx2word.items(), key=lambda x: x[0])]
        self.lookup_table = tf.contrib.lookup.index_to_string_table_from_tensor(self.word_mapping, default_value="<UNK>")
        self.lookup_table.init.run(session=K.get_session())
        self.elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=self.trainable)
        super(ELMoEmbedding, self).build(input_shape)

    def call(self, x):
        x = tf.cast(x, dtype=tf.int64)
        sequence_lengths = tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
        strings = tf.squeeze(self.lookup_table.lookup(x))
        inputs = {
            "tokens": strings,
            "sequence_len": sequence_lengths
        }
        return self.elmo_model(inputs, signature="tokens", as_dict=True)[self.output_mode]

    def compute_output_shape(self, input_shape):
        if self.output_mode == "default":
            return (input_shape[0], 1024)
        if self.output_mode == "word_emb":
            return (input_shape[0], self.max_length, 512)
        if self.output_mode == "lstm_outputs1":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "lstm_outputs2":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "elmo":
            return (input_shape[0], self.max_length, 1024)

    def get_config(self):
        config = {
            'idx2word': self.idx2word,
            'output_mode': self.output_mode 
        }
        return list(config.items())


In [11]:
sentence_input = Input(shape=(X_train.shape[1],), dtype=tf.int64)
sentence_embedding = ELMoEmbedding(idx2word=ind2word, output_mode="elmo", trainable=True)(sentence_input) # These two are interchangeable


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.


W0606 06:59:52.450593 139849357653888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/lookup_ops.py:1318: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use `initializer` instead.


W0606 06:59:52.461754 139849357653888 deprecation.py:323] From <ipython-input-10-316b88bffc30>:21: InitializableLookupTableBase.init (from tensorflow.python.ops.lookup_ops) is deprecated and will be removed after 2018-12-15.
Instructions for updating:
Use `initializer` instead.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 06:59:54.009560 139849357653888 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [12]:
dropout = Dropout(0.5)(sentence_embedding)
lstm_ = LSTM(hidden_size,batch_size=batch_size, return_sequences=True)(sentence_embedding)
timed_ = layers.TimeDistributed(layers.Dense(out_size))(lstm_)
pred = layers.Activation('softmax')(timed_)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0606 07:00:02.056487 139849357653888 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model = Model(inputs=[sentence_input], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 63)                0         
_________________________________________________________________
el_mo_embedding_1 (ELMoEmbed (None, 63, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 63, 32)            135296    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 63, 6)             198       
_________________________________________________________________
activation_1 (Activation)    (None, 63, 6)             0         
Total params: 135,494
Trainable params: 135,494
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Raw test score:', score)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0606 07:00:18.732109 139849357653888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0606 07:00:18.872201 139849357653888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:102: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Train on 1440 samples, validate on 352 samples
Epoch 1/10
1440/1440 [==============================] - 673s 468ms/step - loss: 0.5268 - val_loss: 0.2593
Epoch 2/10
1440/1440 [==============================] - 670s 466ms/step - loss: 0.2068 - val_loss: 0.1704
Epoch 3/10
1440/1440 [==============================] - 658s 457ms/step - loss: 0.1492 - val_loss: 0.1332
Epoch 4/10
1440/1440 [==============================] - 666s 462ms/step - loss: 0.1206 - val_loss: 0.1125
Epoch 5/10
1440/1440 [==============================] - 659s 458ms/step - loss: 0.1032 - val_loss: 0.0996
Epoch 6/10
1440/1440 [==============================] - 664s 461ms/step - loss: 0.0913 - val_loss: 0.0904
Epoch 7/10
1440/1440 [==============================] - 674s 468ms/step - loss: 0.0825 - val_loss: 0.0836
Epoch 8/10
1440/1440 [==============================] - 706s 490ms/step - loss: 0.0756 - val_loss: 0.0783
Epoch 9/10
1440/1440 [==============================] - 661s 459ms/step - loss: 0.0701 - val_loss: 0.0743

In [0]:
def score(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [0]:
y_prob = model.predict(X_train) 
y_classes = y_prob.argmax(axis=-1)

In [18]:
y_classes

array([[0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 0, 0, 4],
       ...,
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 3, 1, 4],
       [0, 0, 0, ..., 4, 3, 4]])

In [19]:
y_classes[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4])

In [0]:
yh = y_train.argmax(2)

In [21]:
pr=y_classes
fyh, fpr = score(yh, pr)
print('Training accuracy:', accuracy_score(fyh, fpr))
print('Training confusion matrix:')
print(confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

Training accuracy: 0.9402818991097923
Training confusion matrix:
[[  857    10    41    81   127]
 [   88   423    31   315   163]
 [   33     3  1124    46    20]
 [   20    53    26 24535   118]
 [  255    30    54   257   946]]


(array([0.6839585 , 0.8150289 , 0.88087774, 0.97229928, 0.68850073]),
 array([0.76792115, 0.41470588, 0.91680261, 0.99123303, 0.61348898]),
 array([0.72351203, 0.5497076 , 0.89848122, 0.98167487, 0.64883402]),
 array([ 1116,  1020,  1226, 24752,  1542]))

In [22]:
y_prob = model.predict(X_test) 
pr = y_prob.argmax(axis=-1)
yh = y_test.argmax(2)
fyh, fpr = score(yh, pr)
print('Testing accuracy:', accuracy_score(fyh, fpr))
print('Testing confusion matrix:')
print(confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

Testing accuracy: 0.9291807718348003
Testing confusion matrix:
[[ 182    3   16   18   49]
 [  32   83    5   93   60]
 [  15    0  242   13   13]
 [   9    9   13 6149   24]
 [  42   10   28   71  206]]


(array([0.65      , 0.79047619, 0.79605263, 0.9692623 , 0.58522727]),
 array([0.67910448, 0.3040293 , 0.85512367, 0.99113475, 0.57703081]),
 array([0.66423358, 0.43915344, 0.82453152, 0.98007651, 0.58110014]),
 array([ 268,  273,  283, 6204,  357]))